# Multiple and Dissociable Effects of Sensory History on Working-Memory Performance

Code for decoding and bias calculation for the associated paper on BioRxiv
https://www.biorxiv.org/content/10.1101/2021.10.31.466639v2

In [27]:
# import packages
import os
import numpy as np
import scipy.io
from wmbias_configs import *
import pycircstat
%load_ext autoreload
%autoreload 2

projectloc = '/Users/jasperhajonides/Documents/EXP8_UpdProtec/'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#imports

fitting_params = scipy.io.loadmat(projectloc + '/data/Fitting_params_Biases.mat')
post_s = scipy.io.loadmat(projectloc + '/MEG/analyses/Channel_selection/posterior_electrodes.mat')
neighb = scipy.io.loadmat(projectloc + '/data/channels/neighbours.mat')


# Run decoding + bias computation

In [35]:
classifier_output = np.zeros((260, 20, 3))
shift = np.zeros((260, 20))
right = np.zeros((10, 260, 20))
left = np.zeros((10, 260, 20))

# Run decoding and bias computation for all subjects.
for sb_count in range(0,20):
  
    # obtain configuration for the selected condition:
    # - 'between_trials'
    # - 'within_trial_update'
    # - 'within_trial_protect'
    config = get_config(sb_count, 'within_trial_protect', size_window = 30)
    
    # obtain decoding evidence for selected condition
    evidence = decoding_function(config, overwrite = True)

    
    # Read in all behavioural data and select the right participant
    df_read = pd.read_csv(projectloc + '/data/behavioural/all_behavioural_May2021.csv')
    df_read = df_read.loc[(df_read['subject'] == sb_count), :]
    
    # Subselect the 
    inx = df_read['stimulus_nr'].isin(config['stimulus_nr'])
    inx2 = df_read[list(config['trial_subselection'].keys())[0]].isin(
        list(config['trial_subselection'].values())[0])[inx]
    evidence['single_trial_evidence_store'] = evidence['single_trial_evidence'][inx2]



    # convolve with cosine and obtain evidence.
    # Compute the classifier evidence for the following angles. 
    for i, name in enumerate(['prev_probe_ang','stim1', 'presented']):
        evidence['single_trial_evidence'] = evidence['single_trial_evidence_store']
        y, evidence['y'] = bin_array(np.array(df_read[name])[inx][inx2],config['nr_bins'])
        evidence = cos_convolve(evidence)
        classifier_output[:,sb_count,i] = evidence['cos_convolved']
    

    
    # get the angular difference between the presented grating and the angle 
    # that is expected to generate the bias (eiter stimulus 1 (within trial 
    # bias) or the probed orientation on the previous trial (between trial 
    # bias))
    df_read['angular_difference'] = pycircstat.cdiff(df_read[config['bias_source']],
                                                      df_read['presented'])
  
    # find the trials that have the appropriate angular difference within range
    # now from the decoding evidence we select trials with an angular 
    # difference that biases orientations to the right/negative, or to the 
    # left/positive. 
    right_diffs = ((df_read['angular_difference'][inx][inx2] >= config['min_deg']/90*np.pi) & 
                   (df_read['angular_difference'][inx][inx2] < config['max_dex']/90*np.pi))
    right[:,:,sb_count] = evidence['single_trial_ev_centered'][right_diffs,:,:].mean(0)
    
    left_diffs = ((df_read['angular_difference'][inx][inx2] <= -config['min_deg']/90*np.pi) &
                  (df_read['angular_difference'][inx][inx2] > -config['max_dex']/90*np.pi))
    left[:,:,sb_count] = evidence['single_trial_ev_centered'][left_diffs,:,:].mean(0)
    
    # the decoding evidence provides us with tuning curves. We like to know 
    # if the tuning curves are systematically shifted to either direction. 
    # to this end we subtract the evidence right from the centre from the 
    # decoding evidence left from the centre. 
    # This will give us a metric of asymmetry or shift in the tuning curve.
    shift[:,sb_count] = ((left[0:4,:,sb_count].mean(0) - 
                          left[5:9,:,sb_count].mean(0)) - 
                         (right[0:4,:,sb_count].mean(0) - 
                          right[5:9,:,sb_count].mean(0))) 


running S01
loaded data
running S02
loaded data
running S03
loaded data
running S04
loaded data
running S05
loaded data
running S06
loaded data
running S07
loaded data
running S08
loaded data
running S09
loaded data
running S10
loaded data
running S11
loaded data
running S12
loaded data
running S13
loaded data
running S14
loaded data
running S15
loaded data
running S16
loaded data
running S17
loaded data
running S18
loaded data
running S19
loaded data
running S20
loaded data
